In [1]:
import os

import papermill as pm

In [2]:
username = ""
source = ""

In [3]:
username = "Fro116"
source = "MAL"
# username = "4209276"
# source = "Training"

In [4]:
data_path = "../../data/recommendations"
if not os.path.exists(data_path):
    os.mkdir(data_path)

In [5]:
papermill_path = f"../../data/recommendations/{username}/papermill/"

In [6]:
def run_ingress_notebook(name):
    outdir = os.path.join(papermill_path, "rec_ingress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    pm.execute_notebook(
        f"../RecIngress/{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source),
    )

## RecEgress

In [20]:
def run_egress_notebook(name):
    outdir = os.path.join(papermill_path, "rec_egress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    pm.execute_notebook(
        f"{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username),
    )

In [8]:
if source == "Training":
    run_egress_notebook("TrainingServingSkew")
else:
    run_egress_notebook("Recommendations")    

Input notebook does not contain a cell with tag 'parameters'


Executing:   0%|          | 0/14 [00:00<?, ?cell/s]

Starting kernel event loops.


In [9]:
print("Success!")

Success!
